# Alpaca Price Data to CSV

In [2]:
import alpaca_trade_api as tradeapi
from alpaca_trade_api.rest import TimeFrame
from datetime import datetime, timedelta
import pandas as pd

API_KEY = "PKN6GO3I5GONGZW9QNS3"
API_SECRET = "aEtyZlL3hvhtews57cmKkZzK13r6CPbg1h9Mf2ei"
BASE_URL = "https://paper-api.alpaca.markets" 

api = tradeapi.REST(API_KEY, API_SECRET, BASE_URL, api_version='v2')

ticker = "QQQ" 
end_date = datetime.now()
start_date = end_date - timedelta(days=365)

bars = api.get_bars(ticker, TimeFrame.Minute, 
                    start_date.strftime('%Y-%m-%dT%H:%M:%SZ'), 
                    end_date.strftime('%Y-%m-%dT%H:%M:%SZ')).df
# Adjust timestamps by subtracting 5 hours since Alpaca data is 5 hours ahead
bars.index = bars.index - timedelta(hours=5)

# Shift index by 5 minutes to create bins that will reflect the previous 5 minutes when shifted back.
bars_shifted = bars.copy()
bars_shifted.index = bars_shifted.index - timedelta(minutes=5)

# Resample on 5-minute intervals to include high, low, close price data
bars_5m = pd.DataFrame()
bars_5m['volume'] = bars_shifted['volume'].resample('5min', label='right', closed='left').sum()
bars_5m['high'] = bars_shifted['high'].resample('5min', label='right', closed='left').max()
bars_5m['low'] = bars_shifted['low'].resample('5min', label='right', closed='left').min()
bars_5m['close'] = bars_shifted['close'].resample('5min', label='right', closed='left').last()
bars_5m['open'] = bars_shifted['open'].resample('5min', label='right', closed='left').first()
bars_5m['vwap'] = (bars_shifted['close'] * bars_shifted['volume'])\
                  .resample('5min', label='right', closed='left').sum() / bars_5m['volume']

# Shift the resampled index back by 5 minutes so that the value labeled 10:00 represents data from 9:55–9:59.
bars_5m.index = bars_5m.index + timedelta(minutes=5)

# Filter for market hours only (9:30 AM to 4:00 PM Eastern Time)
market_hours = bars_5m.between_time('9:30', '16:00')

# Remove the first day to have all data start at the same time, unrelated to time of day
first_date = market_hours.index[0].date()
market_hours = market_hours[market_hours.index.date > first_date]

# Remove all NaN values
market_hours = market_hours.dropna()

print(market_hours.head())


                            volume    high       low   close    open  \
timestamp                                                              
2024-03-05 09:30:00+00:00    42702  441.00  440.6300  441.00  440.70   
2024-03-05 09:35:00+00:00  1967726  440.95  439.4904  439.54  440.94   
2024-03-05 09:40:00+00:00  1423247  439.79  439.0900  439.68  439.52   
2024-03-05 09:45:00+00:00   986065  439.81  439.0600  439.33  439.69   
2024-03-05 09:50:00+00:00  1185201  439.32  438.3100  438.83  439.32   

                                 vwap  
timestamp                              
2024-03-05 09:30:00+00:00  440.859391  
2024-03-05 09:35:00+00:00  440.123686  
2024-03-05 09:40:00+00:00  439.506483  
2024-03-05 09:45:00+00:00  439.459675  
2024-03-05 09:50:00+00:00  438.852920  


Next, we want to split the data into 2 datasets and into a folder, so that we can train our model on the first dataset and verify with recent data. Let's set a parameter 'days_test' to seperate up to the recent 'days_test' days.

In [4]:
import os
from datetime import timedelta

# Create a directory to store the datasets if it doesn't exist
output_dir = f"{ticker}_{end_date.strftime('%Y-%m-%d')}_datasets"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Set the number of days for testing
days_test = 60  # You can adjust this parameter as needed

# Calculate the split date
last_date = market_hours.index[-1].date()
split_date = last_date - timedelta(days=days_test)

# Split the data
train_data = market_hours[market_hours.index.date <= split_date]
test_data = market_hours[market_hours.index.date > split_date]

# Save the datasets to CSV files
train_filename = f"{output_dir}/{ticker}_train_data.csv"
test_filename = f"{output_dir}/{ticker}_test_data.csv"

train_data.to_csv(train_filename)
test_data.to_csv(test_filename)

print(f"Training data saved to {train_filename}, with {len(train_data)} records from {train_data.index[0].date()} to {train_data.index[-1].date()}")
print(f"Testing data saved to {test_filename}, with {len(test_data)} records from {test_data.index[0].date()} to {test_data.index[-1].date()}")
print(f"Split date: {split_date}")

Training data saved to QQQ_2025-03-02_datasets/QQQ_train_data.csv, with 16427 records from 2024-03-05 to 2024-12-30
Testing data saved to QQQ_2025-03-02_datasets/QQQ_test_data.csv, with 3160 records from 2024-12-31 to 2025-02-28
Split date: 2024-12-30
